In [1]:
import pandas as pd
import numpy as np

## 아파트 실거래가 2012년~2022년 데이터 전처리

### csv파일 불러오기

In [2]:
for i in range(2012,2022):
    globals()[f'df{i}'] = pd.read_csv(f'apt_price({i}-{i+1}).csv',encoding='cp949',thousands=',',skiprows=15)
df2012_1 = pd.read_csv('apt_price(2012-1).csv',encoding='cp949',thousands=',',skiprows=15)

In [3]:
%who

df2012	 df2012_1	 df2013	 df2014	 df2015	 df2016	 df2017	 df2018	 df2019	 
df2020	 df2021	 i	 np	 pd	 


### 연도별 csv 합치기

In [4]:
df = pd.concat([df2012_1,df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020, df2021],axis=0)

In [5]:
df = df.reset_index(drop=True)
df

시군구      번지     본번    부번             단지명  전용면적(㎡)    계약년월  \
0       서울특별시 강남구 개포동   655-2  655.0   2.0  개포2차현대아파트(220)    77.75  201205   
1       서울특별시 강남구 개포동   655-2  655.0   2.0  개포2차현대아파트(220)    77.75  201207   
2       서울특별시 강남구 개포동     652  652.0   0.0          개포우성3차   104.43  201204   
3       서울특별시 강남구 개포동    12-2   12.0   2.0      개포자이(12-2)   134.30  201205   
4       서울특별시 강남구 개포동     141  141.0   0.0         개포주공1단지    50.38  201201   
...               ...     ...    ...   ...             ...      ...     ...   
824116  서울특별시 중랑구 중화동     450  450.0   0.0  한신아파트(103~109)    59.76  202207   
824117  서울특별시 중랑구 중화동     450  450.0   0.0  한신아파트(103~109)    84.03  202207   
824118  서울특별시 중랑구 중화동  274-51  274.0  51.0         한영(101)    84.69  202111   
824119  서울특별시 중랑구 중화동  274-51  274.0  51.0         한영(101)    84.69  202204   
824120  서울특별시 중랑구 중화동    11-7   11.0   7.0     현대휴앤미(102동)    76.43  202204   

        계약일  거래금액(만원)   층    건축년도          도로명  해제사유발생일  거래유형          중개사소재지  
0         1     66000   4  1988.0      언주로 103      NaN     -               -  
1         9     58000   2  1988.0      언주로 103      NaN     -               -  
2        13     85000   6  1984.0      개포로 307      NaN     -               -  
3         8    119500  14  2004.0   개포로109길 69      NaN     -               -  
4         9     77500   1  1982.0         개포로       NaN     -               -  
...     ...       ...  ..     ...          ...      ...   ...             ...  
824116   20     74000   3  1997.0      동일로 752      NaN  중개거래          서울 중랑구  
824117   27     91500  12  1997.0      동일로 752      NaN  중개거래          서울 중랑구  
824118    4     50000   2  2003.0   동일로144길 74      NaN  중개거래  서울 송파구, 서울 중랑구  
824119    9     49900   7  2003.0   동일로144길 74      NaN  중개거래          서울 중랑구  
824120   23     53000   6  2007.0  봉화산로27가길 19      NaN  중개거래          서울 중랑구  

[824121 rows x 15 columns]

### 결측치 확인 [번지, 본번, 부번, 건축년도, 해제사유발생일]

In [6]:
df.isnull().sum()

시군구              0
번지             220
본번              74
부번              74
단지명              0
전용면적(㎡)          0
계약년월             0
계약일              0
거래금액(만원)         0
층                0
건축년도             2
도로명              0
해제사유발생일     819175
거래유형             0
중개사소재지           0
dtype: int64

### '건축년도' 결측치 처리 (DMC SK뷰) 2개

In [7]:
df[df['건축년도'].isnull()]

시군구  번지    본번   부번        단지명  전용면적(㎡)    계약년월  계약일  \
822450  서울특별시 은평구 수색동  30  30.0  0.0  DMCSKVIEW    84.99  202201   28   
822451  서울특별시 은평구 수색동  30  30.0  0.0  DMCSKVIEW    84.99  202202   10   

        거래금액(만원)  층  건축년도      도로명  해제사유발생일  거래유형  중개사소재지  
822450    140000  7   NaN  수색로 216      NaN  중개거래  서울 은평구  
822451    137000  6   NaN  수색로 216      NaN  중개거래  서울 은평구

In [8]:
df['건축년도'].fillna(2021,inplace=True)

In [9]:
df['건축년도'] = df['건축년도'].astype('int32')

In [10]:
df.isnull().sum()

시군구              0
번지             220
본번              74
부번              74
단지명              0
전용면적(㎡)          0
계약년월             0
계약일              0
거래금액(만원)         0
층                0
건축년도             0
도로명              0
해제사유발생일     819175
거래유형             0
중개사소재지           0
dtype: int64

### '번지',  '본번' , '부번' 결측치 처리 220개(서초포레스타2단지, 힐스테이트 서초 젠트리스)

### 잘못된 값(우남퍼스트빌더센트럴) 수정

In [11]:
df[df['번지'].isnull()]['단지명'].unique()

array(['서초포레스타2단지', '힐스테이트 서초 젠트리스'], dtype=object)

In [12]:
df[df['번지'].isnull()]

시군구   번지   본번   부번            단지명  전용면적(㎡)    계약년월  계약일  \
236153  서울특별시 서초구 내곡동  NaN  0.0  0.0      서초포레스타2단지    84.48  201506   29   
240148  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스    84.95  201503    1   
240149  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스    84.99  201504   17   
240150  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스   101.90  201505    1   
240151  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스    84.95  201506   16   
...               ...  ...  ...  ...            ...      ...     ...  ...   
786272  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스    84.99  202107   26   
817129  서울특별시 서초구 내곡동  NaN  0.0  0.0      서초포레스타2단지    84.87  202110   13   
817130  서울특별시 서초구 내곡동  NaN  0.0  0.0      서초포레스타2단지    84.41  202111   11   
817131  서울특별시 서초구 내곡동  NaN  0.0  0.0      서초포레스타2단지    84.87  202202    5   
817918  서울특별시 서초구 신원동  NaN  NaN  NaN  힐스테이트 서초 젠트리스   101.90  202206   23   

        거래금액(만원)   층  건축년도          도로명     해제사유발생일  거래유형    중개사소재지  
236153     45779   9  2015     헌릉로8길 45         NaN     -         -  
240148     73430   5  2014  헌릉로8길 10-12         NaN     -         -  
240149     79000   9  2014  헌릉로8길 10-12         NaN     -         -  
240150     95000   6  2014  헌릉로8길 10-12         NaN     -         -  
240151     87200   6  2014  헌릉로8길 10-12         NaN     -         -  
...          ...  ..   ...          ...         ...   ...       ...  
786272    182000   7  2014  헌릉로8길 10-12         NaN     -         -  
817129    178000  10  2015     헌릉로8길 45  20211202.0     -         -  
817130    160000   9  2015     헌릉로8길 45         NaN  중개거래  경기 용인기흥구  
817131    175000  10  2015     헌릉로8길 45         NaN  중개거래    서울 서초구  
817918    204000   5  2014  헌릉로8길 10-12         NaN  중개거래    서울 서초구  

[220 rows x 15 columns]

In [13]:
df.loc[df[df['단지명']=='서초포레스타2단지'].index,'번지'] = 384
df.loc[df[df['단지명']=='서초포레스타2단지'].index,'본번'] = df.loc[df[df['단지명']=='서초포레스타2단지'].index,'번지']
df.loc[df[df['단지명']=='힐스테이트 서초 젠트리스'].index,'번지'] = 557
df.loc[df[df['단지명']=='힐스테이트 서초 젠트리스'].index,'본번'] = df.loc[df[df['단지명']=='힐스테이트 서초 젠트리스'].index,'번지']
df.loc[df[df['단지명']=='힐스테이트 서초 젠트리스'].index,'부번'] = 0.0
df.loc[df[df['단지명']=='우남퍼스트빌더센트럴'].index,'번지'] = '216-1'
df.loc[df[df['단지명']=='우남퍼스트빌더센트럴'].index,'본번'] = 216
df.loc[df[df['단지명']=='우남퍼스트빌더센트럴'].index,:]

시군구     번지   본번   부번         단지명   전용면적(㎡)    계약년월  계약일  \
677036  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴   84.9149  202007   13   
677037  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴  116.9910  202008   10   
766107  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴   84.9149  202010   23   
766108  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴   84.9817  202012   30   
766109  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴   84.9149  202104   30   
812220  서울특별시 구로구 항동  216-1  216  0.0  우남퍼스트빌더센트럴   84.9149  202112   27   

        거래금액(만원)   층  건축년도    도로명  해제사유발생일 거래유형 중개사소재지  
677036     53000  11  2020  항동로 8      NaN    -      -  
677037     87000   8  2020  항동로 8      NaN    -      -  
766107     50000   8  2020  항동로 8      NaN    -      -  
766108     81500  11  2020  항동로 8      NaN    -      -  
766109     87000  11  2020  항동로 8      NaN    -      -  
812220     63000   1  2020  항동로 8      NaN  직거래      -

### '부번' 결측치 체크

In [14]:
df[df['부번'].isnull()]

Empty DataFrame
Columns: [시군구, 번지, 본번, 부번, 단지명, 전용면적(㎡), 계약년월, 계약일, 거래금액(만원), 층, 건축년도, 도로명, 해제사유발생일, 거래유형, 중개사소재지]
Index: []

### '본번', '부번' int32로 타입변경

In [15]:
df[['본번','부번']]= df[['본번','부번']].astype('int32')

In [16]:
df.isnull().sum()

시군구              0
번지               0
본번               0
부번               0
단지명              0
전용면적(㎡)          0
계약년월             0
계약일              0
거래금액(만원)         0
층                0
건축년도             0
도로명              0
해제사유발생일     819175
거래유형             0
중개사소재지           0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799457 entries, 0 to 799456
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       799457 non-null  object 
 1   번지        799237 non-null  object 
 2   본번        799383 non-null  float64
 3   부번        799383 non-null  float64
 4   단지명       799457 non-null  object 
 5   전용면적(㎡)   799457 non-null  float64
 6   계약년월      799457 non-null  int64  
 7   계약일       799457 non-null  int64  
 8   거래금액(만원)  799457 non-null  int64  
 9   층         799457 non-null  int64  
 10  건축년도      799457 non-null  int32  
 11  도로명       799457 non-null  object 
 12  해제사유발생일   4946 non-null    float64
 13  거래유형      799457 non-null  object 
 14  중개사소재지    799457 non-null  object 
dtypes: float64(4), int32(1), int64(4), object(6)
memory usage: 88.4+ MB


### 컬럼 추가('구', '동', '평', '거래금액_억원', '계약년', '계약월', '평형'), 컬럼 삭제('해제사유발생일')

In [24]:
df['구'] = df['시군구'].str.split(' ').str[1]
df['동'] = df['시군구'].str.split(' ').str[2]
df['평'] = round(df['전용면적(㎡)'] / 3.03,1)
df['거래금액_억원'] = df['거래금액(만원)'] / 10000
df['계약년'] = (df['계약년월'] // 100)
df['계약월'] = df['계약년월'] % 100

In [18]:
df = df.drop(columns='해제사유발생일')

In [25]:
df['평형']=pd.cut(
    df['평'],
    bins=[0,10,20,30,40,50,1000],
    labels='10평이하,10평대,20평대,30평대,40평대,50평이상'.split(',')
)

In [27]:
df.head()

시군구     번지   본번  부번             단지명  전용면적(㎡)    계약년월  계약일  \
0  서울특별시 강남구 개포동  655-2  655   2  개포2차현대아파트(220)    77.75  201205    1   
1  서울특별시 강남구 개포동  655-2  655   2  개포2차현대아파트(220)    77.75  201207    9   
2  서울특별시 강남구 개포동    652  652   0          개포우성3차   104.43  201204   13   
3  서울특별시 강남구 개포동   12-2   12   2      개포자이(12-2)   134.30  201205    8   
4  서울특별시 강남구 개포동    141  141   0         개포주공1단지    50.38  201201    9   

   거래금액(만원)   층  ...         도로명 거래유형 중개사소재지    구    동     평  거래금액_억원   계약년  \
0     66000   4  ...     언주로 103    -      -  강남구  개포동  25.7     6.60  2012   
1     58000   2  ...     언주로 103    -      -  강남구  개포동  25.7     5.80  2012   
2     85000   6  ...     개포로 307    -      -  강남구  개포동  34.5     8.50  2012   
3    119500  14  ...  개포로109길 69    -      -  강남구  개포동  44.3    11.95  2012   
4     77500   1  ...        개포로     -      -  강남구  개포동  16.6     7.75  2012   

   계약월    평형  
0    5  20평대  
1    7  20평대  
2    4  30평대  
3    5  40평대  
4    1  10평대  

[5 rows x 21 columns]

In [28]:
df.to_csv('apt_price.csv',index=False)